# 605波动率优化回落控制

In [1]:
from tick_trade_api import TradeAPI
# 输入指定账户
trade_api=TradeAPI('79855043',order_policy=MarketPolicy)


In [2]:
source_code2="""

# 初始化函数
def init(context):
    # #导入模型
    import pandas as pd
    # 运作周期
    #context.runstep = 3
    # 设置基准收益
    set_benchmark('000300.SH')
    #set_benchmark(002060.SZ')
    # 波动差均值系数
    context.N = 60
    # 设置操作标的：
    #context.stockpool = ['002625.SZ', '600657.SH', '002060.SZ', '600359.SH']
    #context.stockpool = ['002060.SZ']
    file_path = '股票大池435_ths.csv'
    context.stockpool = []
    g.stockpool_1 = []
    context.holdstock0 = []
    df =pd.read_csv(file_path)
    g.stockpool_1 = list(df)[1:]
    print('len(g.stockpool_1)=', len(g.stockpool_1))
    #context.n = 30 # 最大持股数
    #运用I问财进行股票筛选
    get_iwencai('未停牌,上市时间超过2年,连续2年ROE大于8,连续2年净利润现金含量大于80,连续2年净资产收益率大于4')
    
    # 设置手续费
    set_commission(PerShare(type='stock',cost=0.00025))
    #设置可变滑点1%，表示买入价为实际价格乘101%，卖出价为实际价格乘99%
    set_slippage(PriceSlippage(0.01))
    # 固定盈利
    context.win = 4 #盈利几个点卖出
    # 单笔买入仓位
    context.buypct = 0.03 # 适合1百万元回测账户
    # 止盈
    context.stopwin = 100 # 先不考虑止盈故设置100
    # 止损
    context.stoploss = 0.02  

    # 设置初持仓
    #set_holding_stocks(dict(zip(context.stockpool,[702200]*len(context.stockpool))))
    set_holding_stocks(dict(zip(context.stockpool,[0]*len(context.stockpool))))
    
    # 记录日内买入日期时间
    context.stocktrade = {}
    for stock in g.stockpool_1:
        context.stocktrade[stock] = []
        
    # 记录波动率数据
    g.aa = 0
    g.bddata = {}
    for stock in g.stockpool_1:
        g.bddata[stock] = []
    
    # 获取当前持仓
    holdstock = list(context.portfolio.stock_account.positions.keys())
    # 查询持仓情况
    market_value = context.portfolio.stock_account.market_value
    total_value = context.portfolio.stock_account.total_value
    stocklist = list(context.portfolio.stock_account.positions)
    print('持仓股票holdstock：', holdstock, '当前持仓市值:', market_value) 
    print('账户总资产:', int(total_value), '持仓股票stocklist:', stocklist)
    
    

# 每个交易日开市前，打印出目前的账户信息.
def before_trading(context):
    g.stocks=context.iwencai_securities    
    holdstock1 = list(context.portfolio.stock_account.positions.keys())
    stock_t = []
    for security in g.stocks:
        if security in g.stockpool_1:
            stock_t.append(security)
    context.stockpool = list(set(stock_t))[::]
    print('len(context.stockpool)=', len(context.stockpool), context.stockpool)
    context.buypct = int(1/(len(context.stockpool)+2)*1000)/2000
    #log.info(context.portfolio)
    
    stock_t = []
    for security in g.stocks:
        if security in holdstock1:
            stock_t.append(security)
    context.holdstock0 = list(set(stock_t))[::]
    print('context.holdstock0:', len(context.holdstock0) ,context.holdstock0)
    
    # 记录日内买入日期时间
    context.stocktrade = {}
    for stock in context.stockpool:
        context.stocktrade[stock] = []    
    
    
    
def handle_bar(context, bar_dict):    
    # 导入Python-talib库
    #import talib
    #import numpy as np
    import pandas as pd
    # 获取当前分钟
    date = get_datetime().strftime('%Y-%m-%d %H:%M:%S')
    time1 = get_datetime().strftime('%H:%M')
    time = get_datetime().strftime('%M')
    
    # 一、判断盈亏条件
    # 1.1 止盈止损
    # 获取当前持仓可交易股票池
    holdstock = list(context.portfolio.stock_account.positions.keys())
    #if context.holdstock0 != []:
        #holdstock = context.holdstock0[::]
    t1 = ['09:31','10:01','10:31','11:01','11:28','13:01','13:31','14:01','14:31','14:53']
    if time=='31' or (time1 in t1):
        for stock in holdstock:   # 循环持仓可交易股票
            # 获取成本价
            cost_basis = context.portfolio.positions[stock].cost_basis
            # 获取数据
            df = history(stock, ['close','high','low'], 60,'1m', True, 'pre', is_panel=1)
            # 提取数据
            close_price = df['close']
            # high_price = df['high']
            # low_price = df['low']
            new_price = df['close'].iloc[-1] #最新收盘价
            
            # 计算指标，单向波动差择时策略
            # 获取开、高、低价格
            data = history(stock, ['open','high','low'], context.N+1, '30m')
            # 计算上行波动
            dif_up = data['high']/data['open']-1
            # 计算下行波动
            dif_down = 1-data['low']/data['open']
            # 计算波动差值
            dif_T = dif_up-dif_down
            # 计算波动差值均值
            dif_ma = pd.rolling_mean(dif_T,context.N)
            # 若均值为正，则买入或持仓
            #if dif_ma.values[-1] > 0 and value==0:
            # 若均值为负，则卖出或空仓
            #if dif_ma.values[-1] < 0 and value> 0:
            
            # 获取可卖股数等持仓情况
            amount = context.portfolio.positions[stock].amount
            available_amount = context.portfolio.positions[stock].available_amount
            
            if available_amount>0 and dif_ma.values[-1] < 0:
                #order_percent(stock, -context.buypct)
                order(stock, -available_amount)
                # 记录这次卖出时间
                context.stocktrade[stock].append(date)
                market_value = context.portfolio.stock_account.market_value
                total_value = context.portfolio.stock_account.total_value
                
                print("卖出价格:{},毛盈利{}%".format(stock,new_price,int(10000*(new_price/cost_basis -1))/100))
                print('==>卖出股票:', stock, available_amount, '股, 持仓股票市值:', market_value ,'账户总资产:', int(total_value))
                # 移除本股票本轮交易时间记录
                context.stocktrade[stock] = []
                # 当日可交易股票池中移除本股票代码
                b = []
                for i in context.holdstock0:
                    if i != stock:
                        b.append(i)
                context.holdstock0 = b[::]
                break
        
        
            if new_price/cost_basis-1>context.stopwin and available_amount>0:
                # 卖出所有股票
                # print('持仓股数:', amount, '可卖股数:', available_amount, "止盈卖出 %s" % (stock), new_price,available_amount, '股')
                order(stock,-available_amount)
                # 移除本股票本轮交易时间记录
                context.stocktrade[stock] = []
                market_value = context.portfolio.stock_account.market_value
                total_value = context.portfolio.stock_account.total_value
                print('==>持仓市值', int(market_value), '账户总资产:', int(total_value))
                # 当日可交易股票池中移除本股票代码
                b = []
                for i in context.holdstock0:
                    if i != stock:
                        b.append(i)
                context.holdstock0 = b[::]
                break
    
            elif new_price/cost_basis-1< -context.stoploss and available_amount>0:
                # 卖出该股全部
                #print('持仓股数:', amount, '可卖股数:', available_amount, "止损卖出 %s" % (stock), new_price, available_amount, '股')
                order(stock,-available_amount)
                # 移除本股票本轮交易时间记录
                context.stocktrade[stock] = [] 
                market_value = context.portfolio.stock_account.market_value
                total_value = context.portfolio.stock_account.total_value
                # stocklist = list(context.portfolio.stock_account.positions)
                print('==>持仓市值', int(market_value), '账户总资产:', int(total_value))
                # 当日可交易股票池中移除本股票代码
                b = []
                for i in context.holdstock0:
                    if i != stock:
                        b.append(i)
                context.holdstock0 = b[::]
                break
            
            # 1.2 达到获利目标后回落卖出 
            # 取交易时间数据
            if len(context.stocktrade[stock])>0:
                d1 = context.stocktrade[stock][0]
                dd1 = d1[:4]+d1[5:7]+d1[8:10]+' '+d1[11:16]
                dd2 = date
                df = get_price(stock, dd1, dd2,'1m', ['close', 'open', 'low', 'high'], skip_paused = False, fq = 'pre', is_panel = 1)
    
                # 计算本轮首次买入后至最新价期间的最高价及最大涨幅%
                # 获取成本价
                cost_basis = context.portfolio.positions[stock].cost_basis
                new_price = df['close'].iloc[-1]
                max_price = df['high'].max()
                max_zf = (max_price-cost_basis)/cost_basis*100
                # 当最大涨幅大于context.win时候
                if max_zf>context.win and max_zf<20:
                    # 计算本轮首次买入后至最新价期间的冲高回落幅度%
                    hlfd = ((max_price-new_price)/(max_price-cost_basis))*100
                    if hlfd > 20 and available_amount>0:
                        # 卖出该股全部
                        print('持仓股数:', amount, '可卖股数:', available_amount, "回落卖出:{},卖出价格:{},毛盈利{}%".format(stock,new_price,int(10000*(new_price/cost_basis -1))/100))
                        order(stock,-available_amount)
                        # 移除本股票本轮交易时间记录
                        context.stocktrade[stock] = []
                        market_value = context.portfolio.stock_account.market_value
                        total_value = context.portfolio.stock_account.total_value
                        print('==>持仓市值', int(market_value), '账户总资产:', int(total_value))
                        # 当日可交易股票池中移除本股票代码
                        b = []
                        for i in context.holdstock0:
                            if i != stock:
                                b.append(i)
                        context.holdstock0 = b[::]
                        break
                        
                # 当最大涨幅大于等于20%时候
                if max_zf>=20:
                    # 计算本轮首次买入后至最新价期间的冲高回落幅度%
                    hlfd = ((max_price-new_price)/(max_price-cost_basis))*100
                    if hlfd > 10 and available_amount>0:
                        # 卖出该股全部
                        print('持仓股数:', amount, '可卖股数:', available_amount, "回落卖出:{},卖出价格:{},毛盈利{}%".format(stock,new_price,int(10000*(new_price/cost_basis -1))/100))
                        order(stock,-available_amount)
                        # 移除本股票本轮交易时间记录
                        context.stocktrade[stock] = []
                        market_value = context.portfolio.stock_account.market_value
                        total_value = context.portfolio.stock_account.total_value
                        stocklist = list(context.portfolio.stock_account.positions)
                        print('==>持仓市值', int(market_value), '账户总资产:', int(total_value))
                        # 当日可交易股票池中移除本股票代码
                        b = []
                        for i in context.holdstock0:
                            if i != stock:
                                b.append(i)
                        context.holdstock0 = b[::]
                        break

    
    # 二、单向波动差择时策略_买入
    if time1=='14:01':
        for stock in context.stockpool:
            # 获取可卖股数等持仓情况
            amount = context.portfolio.positions[stock].amount
            available_amount = context.portfolio.positions[stock].available_amount
            if amount-available_amount>0:
                break
            #获取数据
            df1 = history(stock, ['close'], 60,'1m', True, 'pre', is_panel=1)
            df_d = history(stock, ['open','high','low'], 3,'60m', True, 'pre', is_panel=1)
            # 提取数据
            close_price = df1['close']
            high_price = df_d['high'].max()
            low_price = df_d['low'].min()
            new_price = df1['close'].iloc[-1]
            open_price = df_d['open'].iloc[0]
            
            # 计算指标，单向波动差择时策略
            # 获取开、高、低价格
            data = history(stock, ['open','high','low'], context.N+1, '60m')
            # 计算上行波动
            dif_up = data['high']/data['open']-1
            # 计算下行波动
            dif_down = 1-data['low']/data['open']
            # 计算波动差值
            dif_T = dif_up-dif_down
            # 计算波动差值均值
            dif_ma = pd.rolling_mean(dif_T,context.N+1)
            # 若均值为正，则买入或持仓
            #if dif_ma.values[-1] > 0 and value==0:
            # 若均值为负，则卖出或空仓
            #if dif_ma.values[-1] < 0 and value> 0:
            
            # 最新价的涨幅
            
            new_zf = int((new_price - open_price)/open_price * 10000)/100
            # 获取持仓情况
            market_value = context.portfolio.stock_account.market_value
            total_value = context.portfolio.stock_account.total_value
            total_market = total_value - market_value
            numok = (0.99*total_market)/new_price
    
            if  dif_ma.values[-1] > 0 and numok>200 and new_zf<5:
                order_percent(stock, context.buypct)
                market_value = context.portfolio.stock_account.market_value
                total_value = context.portfolio.stock_account.total_value            
                # 记录这次买入时间
                context.stocktrade[stock].append(date)            
                print('==>买入股票',stock, '涨幅{},最新价{}'.format(new_zf,new_price), '持仓股票市值', market_value , '账户总资产:', int(total_value))
                print('context.stocktrade[stock].append(date),记录这次买入时间==>', context.stocktrade[stock])
                

            
# 收盘后运行函数,用于储存自定义参数、全局变量,执行盘后选股等
def after_trading(context):
    market_value = context.portfolio.stock_account.market_value
    total_value = context.portfolio.stock_account.total_value            
    print('==>持仓股票市值', market_value , '账户总资产:', int(total_value))
    log.info('一天结束')            

"""



In [3]:
# 系统重置
try:
    import shutil
    shutil.rmtree('persist/研究环境策略')
except:
    pass


In [4]:
# 模拟运行
rtrade = research_trade('研究环境策略', source_code2, capital_base=float(7190000), frequency='MINUTE', trade_api=trade_api)#trade_api可传入List[TradeAPI]，实现多账户交易


2022-06-09 09:26:20.365176 - INFO - 开始research_trade运行...
2022-06-09 00:00:00.000000 - INFO - len(g.stockpool_1)= 435
2022-06-09 00:00:00.000000 - INFO - 持仓股票holdstock： [] 当前持仓市值: 0
2022-06-09 00:00:00.000000 - INFO - 账户总资产: 7190000 持仓股票stocklist: []
2022-06-09 09:00:00.000000 - INFO - len(context.stockpool)= 126 ['000818.SZ', '002028.SZ', '600764.SH', '600820.SH', '002270.SZ', '300122.SZ', '300450.SZ', '000906.SZ', '601618.SH', '002050.SZ', '300630.SZ', '603867.SH', '600030.SH', '002508.SZ', '002990.SZ', '002064.SZ', '000682.SZ', '600176.SH', '002925.SZ', '600862.SH', '002651.SZ', '688389.SH', '600422.SH', '601012.SH', '600667.SH', '002916.SZ', '600535.SH', '600436.SH', '600380.SH', '000895.SZ', '002222.SZ', '603060.SH', '300082.SZ', '603970.SH', '600183.SH', '300699.SZ', '002979.SZ', '603987.SH', '601006.SH', '600993.SH', '600740.SH', '000897.SZ', '600997.SH', '601888.SH', '300002.SZ', '600373.SH', '603883.SH', '002241.SZ', '300418.SZ', '603606.SH', '002015.SZ', '600763.SH', '000963.

2022-06-09 14:01:00.000000 - INFO - ==>买入股票 600862.SH 涨幅-5.2,最新价24.02 持仓股票市值 410729.0 账户总资产: 7187840
2022-06-09 14:01:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2022-06-09 14:01:00']
2022-06-09 14:01:00.000000 - WARN - realtime_trade订单(002651.SZ 数量:3300 价格:7.5174)执行失败,失败原因: Symbol 002651.SZ 委托失败: [用户登录]对应客户端未登录，禁止登录
2022-06-09 14:01:00.000000 - INFO - ==>买入股票 002651.SZ 涨幅-0.66,最新价7.48 持仓股票市值 435413.0 账户总资产: 7187710
2022-06-09 14:01:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2022-06-09 14:01:00']
2022-06-09 14:01:00.000000 - WARN - realtime_trade订单(688389.SH 数量:1633 价格:15.3966)执行失败,失败原因: Symbol 688389.SH 委托失败: [用户登录]对应客户端未登录，禁止登录
2022-06-09 14:01:00.000000 - INFO - ==>买入股票 688389.SH 涨幅-3.22,最新价15.32 持仓股票市值 460430.56 账户总资产: 7187579
2022-06-09 14:01:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2022-06-09 14:01:00']
2022-06-09 14:01:00.000000 - WARN - realtime_trade订单(600422.SH 数量:2000 价格:12.3816)执行失败,失败原因

2022-06-09 14:01:00.000000 - INFO - ==>买入股票 002241.SZ 涨幅-2.28,最新价38.0 持仓股票市值 905460.56 账户总资产: 7185236
2022-06-09 14:01:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2022-06-09 14:01:00']
2022-06-09 14:01:00.000000 - WARN - realtime_trade订单(300418.SZ 数量:1600 价格:14.94435)执行失败,失败原因: Symbol 300418.SZ 委托失败: [用户登录]对应客户端未登录，禁止登录
2022-06-09 14:01:00.000000 - INFO - ==>买入股票 300418.SZ 涨幅-1.19,最新价14.87 持仓股票市值 929252.56 账户总资产: 7185111
2022-06-09 14:01:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2022-06-09 14:01:00']
2022-06-09 14:01:00.000000 - WARN - realtime_trade订单(603606.SH 数量:400 价格:56.4006)执行失败,失败原因: Symbol 603606.SH 委托失败: [用户登录]对应客户端未登录，禁止登录
2022-06-09 14:01:00.000000 - INFO - ==>买入股票 603606.SH 涨幅-2.63,最新价56.12 持仓股票市值 951700.56 账户总资产: 7184994
2022-06-09 14:01:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2022-06-09 14:01:00']
2022-06-09 14:01:00.000000 - WARN - realtime_trade订单(600763.SH 数量:100 价格:145.6144499999

2022-06-09 14:01:00.000000 - WARN - realtime_trade订单(300383.SZ 数量:2500 价格:9.828899999999999)执行失败,失败原因: Symbol 300383.SZ 委托失败: [用户登录]对应客户端未登录，禁止登录
2022-06-09 14:01:00.000000 - INFO - ==>买入股票 300383.SZ 涨幅-2.0,最新价9.78 持仓股票市值 1346408.56 账户总资产: 7182911
2022-06-09 14:01:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2022-06-09 14:01:00']
2022-06-09 14:01:00.000000 - WARN - realtime_trade订单(603416.SH 数量:800 价格:30.64245)执行失败,失败原因: Symbol 603416.SH 委托失败: [用户登录]对应客户端未登录，禁止登录
2022-06-09 14:01:00.000000 - INFO - ==>买入股票 603416.SH 涨幅-3.17,最新价30.49 持仓股票市值 1370800.56 账户总资产: 7182783
2022-06-09 14:01:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2022-06-09 14:01:00']
2022-06-09 14:01:00.000000 - WARN - realtime_trade订单(300036.SZ 数量:1300 价格:18.1905)执行失败,失败原因: Symbol 300036.SZ 委托失败: [用户登录]对应客户端未登录，禁止登录
2022-06-09 14:01:00.000000 - INFO - ==>买入股票 300036.SZ 涨幅-2.42,最新价18.1 持仓股票市值 1394330.56 账户总资产: 7182659
2022-06-09 14:01:00.000000 - INFO - context.stocktr

2022-06-09 14:01:00.000000 - INFO - ==>买入股票 300459.SZ 涨幅-2.02,最新价3.38 持仓股票市值 1813369.96 账户总资产: 7180453
2022-06-09 14:01:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2022-06-09 14:01:00']
2022-06-09 14:01:00.000000 - WARN - realtime_trade订单(000786.SZ 数量:800 价格:29.4867)执行失败,失败原因: Symbol 000786.SZ 委托失败: [用户登录]对应客户端未登录，禁止登录
2022-06-09 14:01:00.000000 - INFO - ==>买入股票 000786.SZ 涨幅-0.2,最新价29.34 持仓股票市值 1836841.96 账户总资产: 7180330
2022-06-09 14:01:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2022-06-09 14:01:00']
2022-06-09 14:01:00.000000 - WARN - realtime_trade订单(600761.SH 数量:2400 价格:10.351500000000001)执行失败,失败原因: Symbol 600761.SH 委托失败: [用户登录]对应客户端未登录，禁止登录
2022-06-09 14:01:00.000000 - INFO - ==>买入股票 600761.SH 涨幅-1.24,最新价10.3 持仓股票市值 1861561.96 账户总资产: 7180200
2022-06-09 14:01:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2022-06-09 14:01:00']
2022-06-09 14:01:00.000000 - WARN - realtime_trade订单(002973.SZ 数量:400.0 价格:1

2022-06-07 14:43:00.061358 - INFO - 开始research_trade运行...
2022-06-07 00:00:00.000000 - INFO - len(g.stockpool_1)= 435
2022-06-07 00:00:00.000000 - INFO - 持仓股票holdstock： ['002633.SZ'] 当前持仓市值: 0.0
2022-06-07 00:00:00.000000 - INFO - 账户总资产: 50000 持仓股票stocklist: ['002633.SZ']
2022-06-07 00:00:00.000000 - INFO - 结束research_trade运行...

['002979.SZ', '600309.SH', '300122.SZ', '300481.SZ', '300760.SZ', '300450.SZ', '000906.SZ', '300146.SZ', '002916.SZ', '600398.SH', '000963.SZ', '300412.SZ', '000897.SZ', '601369.SH', '603416.SH', '600176.SH', '688389.SH', '002821.SZ', '601012.SH', '603606.SH', '300575.SZ', '603867.SH', '603128.SH', '600030.SH', '002497.SZ', '600422.SH', '300398.SZ', '000682.SZ', '002056.SZ', '002368.SZ', '002475.SZ', '002064.SZ', '600389.SH', '600976.SH', '300630.SZ', '603901.SH', '600761.SH', '002432.SZ', '601100.SH', '600298.SH', '600085.SH', '002270.SZ', '600373.SH', '300002.SZ', '002938.SZ', '002332.SZ', '002714.SZ', '603926.SH', '600183.SH', '600820.SH', '300059.SZ', '601607.SH', '603127.SH', '002311.SZ', '002415.SZ', '002973.SZ', '300357.SZ', '000895.SZ', '002028.SZ', '600740.SH', '603866.SH', '000818.SZ', '002222.SZ', '300354.SZ', '002829.SZ', '300383.SZ', '603599.SH', '000030.SZ', '002761.SZ', '002050.SZ', '002049.SZ', '600997.SH', '603970.SH', '601888.SH', '600763.SH', '002241.SZ', '300459.SZ', '000786.SZ', '002990.SZ', '300244.SZ', '002645.SZ', '600580.SH', '002724.SZ', '600332.SH', '300547.SZ', '600276.SH', '600380.SH', '000400.SZ', '600862.SH', '603939.SH', '603601.SH', '300418.SZ', '300642.SZ', '603987.SH', '603808.SH', '002130.SZ', '601006.SH', '300036.SZ', '600379.SH', '000848.SZ', '300699.SZ', '600535.SH', '601689.SH', '002015.SZ', '002925.SZ', '600993.SH', '600667.SH', '600546.SH', '601618.SH', '688036.SH', '603060.SH', '600436.SH', '600764.SH', '002508.SZ', '600031.SH', '002481.SZ', '300082.SZ', '002078.SZ', '600285.SH', '603883.SH', '002651.SZ', '002439.SZ', '600233.SH', '300015.SZ', '600885.SH', '002062.SZ']

In [5]:
# 回测运行
# research_strategy(source_code, start_date=None, end_date=None, capital_base=100000, frequency='DAILY', stock_market='STOCK', benchmark=None)
#research_strategy(source_code2, start_date='20220101', end_date='20220606', capital_base=float(10000000), frequency='MINUTE', stock_market='STOCK', benchmark=None)


In [6]:
# t1 = ['09:31','10:01','10:31','11:01','11:28','13:01','13:31','14:01','14:31','14:53']
# t1

from tick_trade_api import TradeAPI
# 输入指定账户
trade_api=TradeAPI('79855043',order_policy=MarketPolicy)

source_code="""


"""

# 系统重置
try:
    import shutil
    shutil.rmtree('persist/研究环境策略')
except:
    pass

# 模拟运行
rtrade = research_trade('研究环境策略', source_code, capital_base=float(10000000), frequency='MINUTE', trade_api=trade_api)#trade_api可传入List[TradeAPI]，实现多账户交易


# 回测运行
# research_strategy(source_code, start_date=None, end_date=None, capital_base=100000, frequency='DAILY', stock_market='STOCK', benchmark=None)
#research_strategy(source_code, start_date='20180901', end_date='20190301', capital_base=float(10000000), frequency='MINUTE', stock_market='STOCK', benchmark=None)


